In [1]:
!pip install pymssql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 5.1 MB/s 


In [2]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from joblib import dump

In [3]:

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

In [8]:

data = pd.read_sql(
    """
    select ticker, date, ret, acc, agr, bm, gma, lev, mom12m, mom1m, mve, roeq, turn, idiovol, beta
    from data
    where date>='2010-01'
    """, 
    conn
)
data = data.dropna()

In [10]:
data["size"] = data.groupby("date").mve.rank(ascending=False)
data = data[data["size"]>=1000]
data['rnk'] = data.groupby("date").ret.rank(pct=True)
data["xret"] = data.groupby("date").ret.apply(lambda x: x - x.median())

In [14]:
string = "acc, agr, bm, gma, lev, mom12m, mom1m, mve, roeq, turn, idiovol, beta"
features = string.split(", ")
X = data[features]

In [17]:
y = data["rnk"]
model = RandomForestRegressor(
  max_depth=4,
  random_state=0
)
model.fit(X, y)
dump(model, "model_rnk4.joblib")

['model_rnk4.joblib']

In [18]:
model = RandomForestRegressor(
  max_depth=6,
  random_state=0
)
model.fit(X, y)
dump(model, "model_rnk6.joblib")

['model_rnk6.joblib']

In [21]:
y = data["xret"]
model = RandomForestRegressor(
  max_depth=4,
  random_state=0
)
model.fit(X, y)
dump(model, "model_ret4.joblib")

['model_ret4.joblib']

In [22]:
model = RandomForestRegressor(
  max_depth=6,
  random_state=0
)
model.fit(X, y)
dump(model, "model_ret6.joblib")

['model_ret6.joblib']